In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
# Read in Gene expression data's all patients ID as variable: data
data = pd.read_csv('...', index_col = 0)
# Read in the study_subject_file created by REMBRANDT_clin_Bento_transf_script
subject = pd.read_csv('...', sep='\t')

In [3]:
data

,SUBJECT_ID
1,900_00_1961
2,900_00_53_32
3,900_00_5308
4,900_00_5315
5,900_00_5316
...,...
534,NT3
535,NT4
536,NT5
537,NT6


In [4]:
subject

,type,study_subject_id,SUBJECT_ID
0,study_subject,study_subject_id_15,860772
1,study_subject,study_subject_id_1,3089254
2,study_subject,study_subject_id_2,3495697
3,study_subject,study_subject_id_3,3497265
4,study_subject,study_subject_id_4,3504050
...,...,...,...
666,study_subject,study_subject_id_667,NT3
667,study_subject,study_subject_id_668,NT4
668,study_subject,study_subject_id_669,NT5
669,study_subject,study_subject_id_670,NT6


In [5]:
# Inner join the two table on subject ID to select patients and add unique ID feild: study_subject_id
df = pd.merge(left=data, right=subject, on='SUBJECT_ID')
df = df.drop('type', axis=1)

In [6]:
# Set the directory where Gene expression data files are stored
rootDir = '...'

In [7]:
# Get the file size
file_name = []
file_size = []
for dirName, subdirList, fileList in os.walk(rootDir):
#    print('Found directory: %s' % dirName)
    for fname in fileList:
        file_path = dirName + '/' + fname
#        print(file_path)
        file_name.append(fname)
        size = os.stat(file_path).st_size
        file_size.append(size)

In [8]:
# Creat a table of the size of each file
file = pd.DataFrame(list(zip(file_name,file_size)),columns=['file_name', 'file_size'])

In [9]:
# Add feilds to table 
df.insert(0, 'file_type', 'text')
df.insert(0, 'file_format', 'tsv')
df.insert(0, 'file_description', 'Gene expression data')
df.insert(0, 'file_status', 'uploaded')
df.insert(0, 'md5sum', df.index)
df.insert(0, 'file_id', 'file_id_' + (df.index + 1).astype(str))
df['file_name'] = 'GeneExp_' + df['file_id'] + '.txt'
df['file_location'] = 's3://bento-files/OPEN_ACCESS/' + df['file_name']
df['study_subject.study_subject_id'] = df['study_subject_id']
df = df.drop(['SUBJECT_ID', 'study_subject_id'], axis=1)
df.insert(0, 'type', 'file')

In [10]:
# Merge size to table
df = pd.merge(left=df, right=file, on='file_name')

In [11]:
# Final table for File node
df

,type,file_id,md5sum,file_status,file_description,file_format,file_type,file_name,file_location,study_subject.study_subject_id,file_size
0,file,file_id_1,0,uploaded,Gene expression data,tsv,text,GeneExp_file_id_1.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_1...,study_subject_id_18,459827
1,file,file_id_2,1,uploaded,Gene expression data,tsv,text,GeneExp_file_id_2.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_2...,study_subject_id_37,459843
2,file,file_id_3,2,uploaded,Gene expression data,tsv,text,GeneExp_file_id_3.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_3...,study_subject_id_19,459797
3,file,file_id_4,3,uploaded,Gene expression data,tsv,text,GeneExp_file_id_4.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_4...,study_subject_id_20,459817
4,file,file_id_5,4,uploaded,Gene expression data,tsv,text,GeneExp_file_id_5.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_5...,study_subject_id_21,459891
...,...,...,...,...,...,...,...,...,...,...,...
533,file,file_id_534,533,uploaded,Gene expression data,tsv,text,GeneExp_file_id_534.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_5...,study_subject_id_667,459920
534,file,file_id_535,534,uploaded,Gene expression data,tsv,text,GeneExp_file_id_535.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_5...,study_subject_id_668,459884
535,file,file_id_536,535,uploaded,Gene expression data,tsv,text,GeneExp_file_id_536.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_5...,study_subject_id_669,459897
536,file,file_id_537,536,uploaded,Gene expression data,tsv,text,GeneExp_file_id_537.txt,s3://bento-files/OPEN_ACCESS/GeneExp_file_id_5...,study_subject_id_670,459872


In [15]:
df.to_csv('../Clinical/output/file_file.txt', sep = '\t', index = False)